# Examples of CORDEX-CMIP6 cmorization

This notebook should show some examples of how a cmorized CORDEX-CMIP6 dataset could look like.

In [ ]:
!ls ../Tables

## Controlled Vocabulary

The current `CORDEX-CMIP6_CV.json` file contains the controlled vocabulary used by the CMOR3 library for rewriting the model output. Let's have a look:

In [ ]:
import json
import pprint

from IPython.display import JSON

# Opening JSON file
with open("../Tables/CORDEX-CMIP6_CV.json") as json_file:
    CV = json.load(json_file)["CV"]

# JSON(CV['required_global_attributes'])
pprint.pprint(CV["required_global_attributes"])

## Cmorization example

We use the `pyremo.cmor` module to create an example of a CORDEX-CMIP6 dataset.

In [ ]:
import os

import xarray as xr
from cordex import cmor as cmor
import cordex as cx

table_dir = "../Tables"


def test_cmorizer_fx():
    ds = cx.cordex_domain("EUR-11", dummy="topo")
    filename = cmor.cmorize_variable(
        ds,
        "orog",
        mapping_table={"orog": {"varname":"topo"}},
        cmor_table=os.path.join(table_dir, "CORDEX-CMIP6_fx.json"),
        dataset_table=os.path.join(table_dir, "CORDEX-CMIP6_remo_example.json"),
        grids_table=os.path.join(table_dir, "CORDEX-CMIP6_grids.json"),
        CORDEX_domain="EUR-11",
        time_units=None,
        allow_units_convert=True,
    )
    return filename

In [ ]:
filename = test_cmorizer_fx()

Let's have a look at the filename.

In [ ]:
filename

In [ ]:
!ncdump -h $filename

The xarray dataset representation allows to explore the dataset interactively.

In [ ]:
ds = xr.open_dataset(filename)
ds